# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [17]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import geoviews

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tzaneen,-23.8332,30.1635,19.26,91,73,0.73,ZA,1702004976
1,1,port-aux-francais,-49.3500,70.2167,4.49,85,97,15.54,TF,1702004976
2,2,naze,28.3667,129.4833,17.57,58,0,2.53,JP,1702004976
3,3,severo-yeniseyskiy,60.3725,93.0408,-31.90,93,100,1.34,RU,1702004977
4,4,port alfred,-33.5906,26.8910,16.77,87,11,1.34,ZA,1702004977


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_city_data_df = city_data_df.copy()
narrow_city_data_df = narrow_city_data_df.loc[(city_data_df["Max Temp"]<=27) & (city_data_df["Max Temp"] > 21)]
narrow_city_data_df = narrow_city_data_df.loc[city_data_df["Wind Speed"] < 4.5]
narrow_city_data_df =narrow_city_data_df.loc[city_data_df["Cloudiness"] == 0]
narrow_city_data_df

# Drop any rows with null values
narrow_city_data_df = narrow_city_data_df.dropna() 

# Display sample data
narrow_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
88,88,bonthe,7.5264,-12.5050,25.55,84,0,1.11,SL,1702005043
134,134,sur,22.5667,59.5289,21.94,50,0,0.93,OM,1702005056
140,140,porto velho,-8.7619,-63.9039,26.03,83,0,0.51,BR,1702004926
160,160,cabo san lucas,22.8909,-109.9124,25.28,66,0,3.60,MX,1702004579
289,289,acajutla,13.5928,-89.8275,24.89,71,0,2.32,SV,1702005127
304,304,kerikeri,-35.2268,173.9474,22.73,57,0,0.45,NZ,1702005091
350,350,sao mateus,-18.7161,-39.8589,22.12,95,0,1.85,BR,1702005180
437,437,la cruz,23.9167,-106.9000,21.22,67,0,2.64,MX,1702005234
513,513,gunjur,13.2019,-16.7339,22.04,68,0,3.09,GM,1702005265
525,525,tshabong,-26.0500,22.4500,22.21,17,0,3.46,BW,1702005269


### Step 3: Create a new DataFrame called `hotel_df`.

In [18]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_city_data_df.copy()
hotel_df = narrow_city_data_df[["City","Country","Lat","Lng","Humidity"]]
#hotel_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
88,bonthe,SL,7.5264,-12.5050,84,
134,sur,OM,22.5667,59.5289,50,
140,porto velho,BR,-8.7619,-63.9039,83,
160,cabo san lucas,MX,22.8909,-109.9124,66,
289,acajutla,SV,13.5928,-89.8275,71,
304,kerikeri,NZ,-35.2268,173.9474,57,
350,sao mateus,BR,-18.7161,-39.8589,95,
437,la cruz,MX,23.9167,-106.9000,67,
513,gunjur,GM,13.2019,-16.7339,68,
525,tshabong,BW,-26.0500,22.4500,17,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
limit = 10
categories = "accommodation.hotel"
params = {
    "categories" : categories,
    "limit" : limit,
    "apiKey" : geoapify_key,
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
      # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
     # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bonthe - nearest hotel: No hotel found
sur - nearest hotel: Sur Hotel
porto velho - nearest hotel: Hotel Slaviero Essential Porto Velho
cabo san lucas - nearest hotel: Comfort Rooms
acajutla - nearest hotel: Hotel Lara
kerikeri - nearest hotel: Avalon Resort
sao mateus - nearest hotel: Grande Hotel
la cruz - nearest hotel: Hotel Las Palmas
gunjur - nearest hotel: Footsteps Eco Lodge
tshabong - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
88,bonthe,SL,7.5264,-12.5050,84,No hotel found
134,sur,OM,22.5667,59.5289,50,Sur Hotel
140,porto velho,BR,-8.7619,-63.9039,83,Hotel Slaviero Essential Porto Velho
160,cabo san lucas,MX,22.8909,-109.9124,66,Comfort Rooms
289,acajutla,SV,13.5928,-89.8275,71,Hotel Lara
304,kerikeri,NZ,-35.2268,173.9474,57,Avalon Resort
350,sao mateus,BR,-18.7161,-39.8589,95,Grande Hotel
437,la cruz,MX,23.9167,-106.9000,67,Hotel Las Palmas
513,gunjur,GM,13.2019,-16.7339,68,Footsteps Eco Lodge
525,tshabong,BW,-26.0500,22.4500,17,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_hotel


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)